# Market Price Comparison Analysis

# Part 2: Market Basket Analysis

In [130]:
import pandas as pd
import numpy as np
from mlxtend.frequent_patterns import apriori, association_rules
import random
from collections import Counter
import sys
import locale

In [131]:
# Set locale to handle Turkish characters
locale.setlocale(locale.LC_ALL, 'tr_TR.UTF-8')

# Read the original dataset with proper encoding
market_data = pd.read_excel("Market_Price_Comparison_2025.xlsx")

# Remove leading and trailing spaces from column names
market_data.columns = market_data.columns.str.strip()

# Drop unnecessary columns
market_data = market_data.drop(columns=['Supermarket A Info', 'Supermarket B Info'])

In [132]:
market_data

,Category,Supermarket A Product,Supermarket A Price,Supermarket B Product,Supermarket B Price,Price Difference,Percentage Difference
0,basic_food,Reis Yeşil Mercimek 1 Kg,129.95,Reis Yeşil Mercimek 1 Kg,191.95,62.00,47.7
1,basic_food,Hasata Yozgat Sultani Yeşil Mercimek 1 Kg,135.95,Hasata Yozgat Sultani Yeşil Mercimek 1 kg,99.90,36.05,36.1
2,basic_food,Reis Kırmızı Mercimek 1 Kg,140.95,Reis Kırmızı Mercimek 1 Kg,140.95,0.00,0.0
3,basic_food,Reis Sarı Mercimek 1 Kg,222.95,Reis Sarı Mercimek 1 Kg,222.95,0.00,0.0
4,basic_food,Reis Royal Beluga Mercimeği 500 G,182.95,Reis Royal Beluga Mercimeği 500 Gr,182.95,0.00,0.0
...,...,...,...,...,...,...,...
1335,basic_food,Heinz Hot Chilli Sos 245 G,84.95,Heinz Hot Chilli Sos 245 Gr,84.90,0.05,0.1
1336,basic_food,Kühne Teriyaki Sos 250 Ml,144.95,Kühne Teriyaki Sos 250 ml,146.90,1.95,1.3
1337,healthy_living,Kikkoman Organik Soya Sosu 150 Ml,304.95,Kikkoman Organik Soya Sosu 150 ml,324.90,19.95,6.5
1338,basic_food,Tat Hardal 230 G,79.95,Tat Hardal 230 G,59.93,20.02,33.4


In [139]:
# Part 1: Simulate Customer Shopping Baskets
print("Part 1: Generating Customer Shopping Baskets...")

# Create a list of all products with their categories
all_products = []
product_categories = {}
for idx, row in market_data.iterrows():
    category = row['Category']
    product_categories[row['Supermarket A Product']] = category
    product_categories[row['Supermarket B Product']] = category
    all_products.extend([(row['Supermarket A Product'], category), 
                        (row['Supermarket B Product'], category)])

# Calculate product frequencies for probability weighting
product_freq = Counter([p[0] for p in all_products])
total_products = len(all_products)
product_probs = {prod: count/total_products for prod, count in product_freq.items()}

# Define common category combinations (e.g., breakfast items, cleaning supplies)
category_combinations = {
    'breakfast': ['breakfast_products', 'dairy_products', 'bakery'],
    'cleaning': ['cleaning_detergent', 'paper_cosmetics'],
    'snacks': ['snacks', 'beverages'],
    'cooking': ['basic_food', 'meat_poultry_fish', 'fruit_vegetables'],
    'health': ['healthy_living', 'dairy_products']
}

# Generate 10,000 customer baskets
n_customers = 10000
baskets = []

for _ in range(n_customers):
    # Generate basket size (min 1, max 25, average 5)
    basket_size = min(25, max(1, int(np.random.normal(5, 2))))
    
    # Decide on a shopping theme (20% chance of themed basket)
    if random.random() < 0.2:
        # Select a random theme
        theme = random.choice(list(category_combinations.keys()))
        categories = category_combinations[theme]
        
        # Get products from these categories
        theme_products = [p[0] for p in all_products if p[1] in categories]
        if theme_products:
            # Select products based on theme
            basket = random.choices(
                theme_products,
                weights=[product_probs[p] for p in theme_products],
                k=min(basket_size, len(theme_products))
            )
        else:
            # Fallback to regular selection if no theme products
            basket = random.choices(
                list(product_probs.keys()),
                weights=list(product_probs.values()),
                k=basket_size
            )
    else:
        # Regular basket selection
        basket = random.choices(
            list(product_probs.keys()),
            weights=list(product_probs.values()),
            k=basket_size
        )
    
    baskets.append(basket)

# Save the baskets to Excel
try:
    # Convert baskets to DataFrame
    basket_data = []
    for i, basket in enumerate(baskets):
        for item in basket:
            basket_data.append({
                'Basket_ID': i + 1,
                'Item': item,
                'Category': product_categories.get(item, 'Unknown')
            })
    
    basket_df = pd.DataFrame(basket_data)
    
    # Save as Excel file
    basket_df.to_excel('customer_baskets.xlsx', index=False, engine='openpyxl')
    print("\nCustomer baskets have been saved to 'customer_baskets.xlsx'")
except Exception as e:
    print(f"\nError saving to Excel: {str(e)}")
    # Fallback to CSV if Excel fails
    basket_df.to_csv('customer_baskets.csv', index=False, encoding='utf-8')
    print("Customer baskets have been saved to 'customer_baskets.csv'")

# Display basket statistics and sample contents
print("\nBasket Statistics:")
print(f"Total number of baskets: {len(baskets)}")
print(f"Average basket size: {np.mean([len(basket) for basket in baskets]):.2f} items")
print(f"Minimum basket size: {min([len(basket) for basket in baskets])} items")
print(f"Maximum basket size: {max([len(basket) for basket in baskets])} items")

# Show 5 sample baskets
print("\nSample Baskets (5 random examples):")
sample_indices = np.random.choice(len(baskets), 5, replace=False)
for idx, basket_idx in enumerate(sample_indices, 1):
    print(f"\nBasket {idx}:")
    print(f"Number of items: {len(baskets[basket_idx])}")
    print("Items:")
    for item in baskets[basket_idx]:
        print(f"  - {item}")

# Show most common items across all baskets
print("\nMost Common Items in All Baskets:")
all_items = [item for basket in baskets for item in basket]
item_counts = Counter(all_items)
for item, count in item_counts.most_common(10):
    print(f"{item}: {count} times")

Part 1: Generating Customer Shopping Baskets...

Customer baskets have been saved to 'customer_baskets.xlsx'

Basket Statistics:
Total number of baskets: 10000
Average basket size: 4.54 items
Minimum basket size: 1 items
Maximum basket size: 12 items

Sample Baskets (5 random examples):

Basket 1:
Number of items: 1
Items:
  - Knorr Baharat Serisi Pul Biber 65 G

Basket 2:
Number of items: 1
Items:
  - Sinangil Un 1 Kg

Basket 3:
Number of items: 3
Items:
  - Lipton Earl Grey Dökme Çay 1000 G
  - Lipton Ice Tea Limon Pet 1 L
  - Master Nut İç Yer Fıstığı Tuzlu 160 G

Basket 4:
Number of items: 5
Items:
  - Activex Antibakteriyel Hassas Islak Havlu 50'li
  - Activia Doğal Probiyotikli Ananaslı Yoğurt 4x100 g
  - Maxi Bulmaca Cilt
  - Ülker Çokomel Marshmallow Çilekli 36 G
  - La Vache Qui Rit 16lı Üçgen Peynir 200g

Basket 5:
Number of items: 3
Items:
  - Hero Goodies Muz Kakao Meyve Bar 30 G
  - Reis Kırmızı Mercimek 1 Kg
  - Hipp 1 Organik Keçi Sütü Bazlı Bebek Sütü 400 G

Most Common

In [134]:
# Part 2: Convert to Transaction Format and Apply Apriori
print("\nPart 2: Analyzing Association Rules...")

# Convert baskets to one-hot encoded format
unique_products = list(set([item for basket in baskets for item in basket]))
basket_df = pd.DataFrame([[1 if item in basket else 0 for item in unique_products] 
                         for basket in baskets],
                        columns=unique_products)

# Convert to boolean type to avoid deprecation warning
basket_df = basket_df.astype(bool)

# Calculate item frequencies
item_frequencies = basket_df.sum() / len(basket_df)
print("\nItem Frequencies by Category (top 5 items per category):")
for category in market_data['Category'].unique():
    category_items = [item for item in item_frequencies.index if product_categories.get(item) == category]
    if category_items:
        print(f"\n{category}:")
        print(item_frequencies[category_items].sort_values(ascending=False).head())

# Select only the most common items (top 200) to reduce memory usage
top_items = item_frequencies.nlargest(200).index
basket_df_reduced = basket_df[top_items]

print(f"\nAnalyzing {len(top_items)} most common items...")

# Apply Apriori algorithm with lower support threshold
# Support threshold: items must appear in at least 0.05% of baskets
# Confidence threshold: rules must be correct at least 5% of the time
frequent_itemsets = apriori(basket_df_reduced, 
                           min_support=0.0005,  # Lowered to 0.05%
                           use_colnames=True,
                           max_len=3)  # Limit itemset size to 3 items

print(f"\nNumber of frequent itemsets found: {len(frequent_itemsets)}")

if len(frequent_itemsets) > 0:
    # Show the frequent itemsets with their categories
    print("\nFrequent Itemsets (sorted by support):")
    itemsets_with_categories = []
    for idx, row in frequent_itemsets.iterrows():
        items = list(row['itemsets'])
        categories = [product_categories.get(item, 'Unknown') for item in items]
        itemsets_with_categories.append({
            'items': items,
            'categories': categories,
            'support': row['support']
        })
    
    # Sort by support and show top 20
    itemsets_with_categories.sort(key=lambda x: x['support'], reverse=True)
    for idx, itemset in enumerate(itemsets_with_categories[:20], 1):
        print(f"\nCombination {idx}:")
        print("Items:", ', '.join(itemset['items']))
        print("Categories:", ', '.join(itemset['categories']))
        print(f"Support: {itemset['support']:.2%}")
    
    # Generate association rules with lower confidence threshold
    rules = association_rules(frequent_itemsets, 
                            metric="confidence",
                            min_threshold=0.05)  # 5% confidence threshold

    print(f"\nNumber of association rules found: {len(rules)}")

    if len(rules) > 0:
        # Sort rules by lift to find the most interesting ones
        rules = rules.sort_values('lift', ascending=False)


Part 2: Analyzing Association Rules...

Item Frequencies by Category (top 5 items per category):

basic_food:
Knorr Çıtır Pane Harcı 90 G                     0.0092
Ülker Pötibör Bisküvi Çifte Kavrulmuş 175 Gr    0.0052
Bağdat Hindistan Cevizi 50 G                    0.0049
Bağdat Mahlep 30 G                              0.0045
Duru Kepekli Pilavlık Bulgur 1 Kg               0.0045
dtype: float64

ready_meals_frozen_products:
Superfresh Falafel 450 G           0.0037
Superfresh Patates Kroket 450 G    0.0032
Feast Parmak Patates 1 Kg          0.0031
Pınar Piliç Kroket 430 G           0.0029
Pınar Gurme Burger 450 G           0.0029
dtype: float64

meat_poultry_fish:
Banvit Piliç Schnitzel 300 Gr            0.0058
Banvit Piliç Schnitzel 300 G             0.0047
Gurvita İlikli Kemik Suyu Sade 320 Ml    0.0047
Banvit Piliç Schnitzel 700 Gr            0.0046
Banvit Piliç Nuget 300 Gr                0.0043
dtype: float64

healthy_living:
City Farm Organik Keçiboynuzu Pekmezi 450 G         

In [138]:
if len(rules) > 0:
    # Sort rules by lift to find the most interesting ones
    rules = rules.sort_values('lift', ascending=False)

    # Convert frozensets to readable strings before saving
    rules['antecedents'] = rules['antecedents'].apply(lambda x: ', '.join(list(x)))
    rules['consequents'] = rules['consequents'].apply(lambda x: ', '.join(list(x)))

    # Part 3: Present and Explain Rules
    print("\nPart 3: Top 5 Most Meaningful Association Rules:")
    print("\nRule Analysis and Business Recommendations:")

    # Select top 5 rules
    top_rules = rules.head(5)

    for idx, rule in top_rules.iterrows():
        # Get the items from the converted strings
        antecedents = rule['antecedents'].split(', ')
        consequents = rule['consequents'].split(', ')
        confidence = rule['confidence']
        lift = rule['lift']
        support = rule['support']
        
        # Get categories for antecedents and consequents
        antecedent_categories = [product_categories.get(item, 'Unknown') for item in antecedents]
        consequent_categories = [product_categories.get(item, 'Unknown') for item in consequents]
        
        print(f"\nRule {idx + 1}:")
        print(f"If {rule['antecedents']} ({', '.join(antecedent_categories)})")
        print(f"Then {rule['consequents']} ({', '.join(consequent_categories)})")
        print(f"Support: {support:.2%}")
        print(f"Confidence: {confidence:.2%}")
        print(f"Lift: {lift:.2f}")
        
        # Business recommendations based on the rule
        print("\nBusiness Recommendations:")
        print("1. Cross-Promotion Strategy:")
        print(f"   - Place {rule['antecedents']} and {rule['consequents']} in close proximity")
        print(f"   - Create bundle offers combining these products")
        
        print("2. Pricing Strategy:")
        print(f"   - Consider dynamic pricing for {rule['antecedents']} to increase sales of {rule['consequents']}")
        print(f"   - Use {rule['antecedents']} as loss leaders to drive sales of {rule['consequents']}")
        
        print("3. Marketing Strategy:")
        print(f"   - Target customers who buy {rule['antecedents']} with promotions for {rule['consequents']}")
        print(f"   - Create email campaigns highlighting the complementary nature of these products")
        
        print("4. Inventory Management:")
        print(f"   - Ensure {rule['consequents']} are well-stocked when {rule['antecedents']} are on promotion")
        print(f"   - Monitor stock levels of both products to prevent out-of-stock situations")

    # Save the rules to a CSV file with proper encoding
    try:
        # Save as Excel file instead of CSV for better character handling
        rules.to_excel('association_rules.xlsx', index=False, engine='openpyxl')
        print("\nAssociation rules have been saved to 'association_rules.xlsx'")
    except Exception as e:
        print(f"\nError saving to Excel: {str(e)}")
        # Fallback to CSV if Excel fails
        rules.to_csv('association_rules.csv', index=False, encoding='utf-8')
        print("Association rules have been saved to 'association_rules.csv'")

else:
    print("\nNo association rules found with current confidence threshold.")
    print("Consider lowering the confidence threshold further or analyzing the frequent itemsets directly.")


Part 3: Top 5 Most Meaningful Association Rules:

Rule Analysis and Business Recommendations:

Rule 100:
If İçim Yarım Yağlı Üçgen Peynir 100 G (dairy_products)
Then Pınar Protein Kakaolu Süt 500 Ml (dairy_products)
Support: 0.06%
Confidence: 10.91%
Lift: 16.04

Business Recommendations:
1. Cross-Promotion Strategy:
   - Place İçim Yarım Yağlı Üçgen Peynir 100 G and Pınar Protein Kakaolu Süt 500 Ml in close proximity
   - Create bundle offers combining these products
2. Pricing Strategy:
   - Consider dynamic pricing for İçim Yarım Yağlı Üçgen Peynir 100 G to increase sales of Pınar Protein Kakaolu Süt 500 Ml
   - Use İçim Yarım Yağlı Üçgen Peynir 100 G as loss leaders to drive sales of Pınar Protein Kakaolu Süt 500 Ml
3. Marketing Strategy:
   - Target customers who buy İçim Yarım Yağlı Üçgen Peynir 100 G with promotions for Pınar Protein Kakaolu Süt 500 Ml
   - Create email campaigns highlighting the complementary nature of these products
4. Inventory Management:
   - Ensure Pınar P